In [1]:
#API connection and dependency install
import os
from flipside import Flipside
from dotenv import find_dotenv, load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Confirm and define dotenv path
dotenv_path = find_dotenv()
print(f"Loading .env from: {dotenv_path}")

#load env data from path
load_dotenv(dotenv_path, override=True)

#store API_KEY from dotenv file
import os
API_KEY = os.getenv("FLIP_API_KEY")
# print(f"API Key loaded: {API_KEY}")

#instantiate flipside client
flipside = Flipside(API_KEY, "https://api-v2.flipsidecrypto.xyz")

Loading .env from: c:\Users\savan\Desktop\portfolio_repo\arb_airdrop\.env


In [12]:
import os

os.makedirs('data/processed/transfers_batches', exist_ok=True)


Load airdrop recipient wallets as a list. Query wallet transfers in batches from ez_token_transfers

In [13]:
recipients_df = pd.read_csv(r"C:\Users\savan\Desktop\portfolio_repo\arb_airdrop\data\processed\arb_drop_data")

batch_size = 1000
wallet_list = recipients_df['user_address'].unique()

for i in range(0, len(wallet_list), batch_size):
    batch = wallet_list[i:i+batch_size].tolist()
    batch_str = "','".join(batch)
    
    sql = f"""
        SELECT block_number, block_timestamp, contract_address, from_address, to_address, amount
        FROM arbitrum.core.ez_token_transfers
        WHERE (from_address IN ('{batch_str}') OR to_address IN ('{batch_str}'))
        AND contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    """
    
    wallet_transfers = flipside.query(sql)
    
    columns = wallet_transfers.columns
    rows = wallet_transfers.rows  
    wallet_transfers_df = pd.DataFrame(rows, columns=columns)
    wallet_transfers_df.to_parquet(f"data/processed/transfers_batches/transfers_batch_{i}.parquet", index=False)
    print(f"Saved batch {i} to {i+batch_size} with {len(wallet_transfers_df)} rows")

Saved batch 0 to 1000 with 12159 rows
Saved batch 1000 to 2000 with 100000 rows
Saved batch 2000 to 3000 with 10183 rows
Saved batch 3000 to 4000 with 12103 rows
Saved batch 4000 to 5000 with 22875 rows
Saved batch 5000 to 6000 with 10072 rows
Saved batch 6000 to 7000 with 15367 rows
Saved batch 7000 to 8000 with 9428 rows
Saved batch 8000 to 9000 with 8871 rows
Saved batch 9000 to 10000 with 9360 rows
Saved batch 10000 to 11000 with 8172 rows
Saved batch 11000 to 12000 with 7735 rows
Saved batch 12000 to 13000 with 18951 rows
Saved batch 13000 to 14000 with 8089 rows
Saved batch 14000 to 15000 with 14336 rows
Saved batch 15000 to 16000 with 8362 rows
Saved batch 16000 to 17000 with 7140 rows
Saved batch 17000 to 18000 with 8973 rows
Saved batch 18000 to 19000 with 13986 rows
Saved batch 19000 to 20000 with 11196 rows
Saved batch 20000 to 21000 with 11072 rows
Saved batch 21000 to 22000 with 11750 rows
Saved batch 22000 to 23000 with 10164 rows
Saved batch 23000 to 24000 with 13286 row

Stitch together files

In [14]:
import glob

In [18]:
batch_files = glob.glob("notebooks/data/transfers_batches/transfers_batch_*.parquet")
transfers_dfs = [pd.read_parquet(f) for f in batch_files]
transfers_df = pd.concat(transfers_dfs, ignore_index=True)
print(f"Combined all transfer batches: {len(transfers_df)} rows")

ValueError: No objects to concatenate